In [ ]:
import numpy as np

class ACOChargingStations:
    def __init__(self, candidate_stations, demand_points, n_stations_to_select,
                 n_ants=20, n_iterations=50, alpha=1, beta=2, rho=0.1):
        """
        candidate_stations: list of (x,y) tuples for potential station locations
        demand_points: list of (x,y) tuples for EV user demand locations
        n_stations_to_select: how many stations to choose
        """
        self.candidates = np.array(candidate_stations)
        self.demands = np.array(demand_points)
        self.n_select = n_stations_to_select
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha  # pheromone importance
        self.beta = beta    # heuristic importance
        self.rho = rho      # evaporation rate

        self.n_candidates = len(candidate_stations)
        self.pheromone = np.ones(self.n_candidates)
        self.heuristic = self.compute_heuristic()

    def compute_heuristic(self):
        # Heuristic: how good is a candidate station based on average distance to demands (inverse)
        heur = []
        for station in self.candidates:
            dists = np.linalg.norm(self.demands - station, axis=1)
            avg_dist = np.mean(dists)
            heur.append(1.0 / (avg_dist + 1e-6))  # avoid divide by zero
        return np.array(heur)

    def construct_solution(self):
        solution = []
        pheromone = self.pheromone ** self.alpha
        heuristic = self.heuristic ** self.beta
        probs = pheromone * heuristic
        probs = probs / probs.sum()

        # Select n stations probabilistically without replacement
        selected = set()
        candidates_left = set(range(self.n_candidates))
        while len(selected) < self.n_select:
            chosen = np.random.choice(list(candidates_left), p=probs[list(candidates_left)]/probs[list(candidates_left)].sum())
            selected.add(chosen)
            candidates_left.remove(chosen)
        return list(selected)

    def evaluate_solution(self, solution):
        # Compute average distance from each demand point to its nearest selected station
        selected_stations = self.candidates[solution]
        total_dist = 0
        for demand in self.demands:
            dists = np.linalg.norm(selected_stations - demand, axis=1)
            total_dist += np.min(dists)
        avg_dist = total_dist / len(self.demands)

        return -avg_dist

    def update_pheromone(self, all_solutions, all_scores):
        self.pheromone *= (1 - self.rho)  # evaporation
        for sol, score in zip(all_solutions, all_scores):
            for station_idx in sol:
                self.pheromone[station_idx] += max(0, score)  # deposit pheromone

    def run(self):
        best_solution = None
        best_score = -float('inf')

        for iteration in range(self.n_iterations):
            all_solutions = []
            all_scores = []

            for _ in range(self.n_ants):
                sol = self.construct_solution()
                score = self.evaluate_solution(sol)
                all_solutions.append(sol)
                all_scores.append(score)

                if score > best_score:
                    best_score = score
                    best_solution = sol

            self.update_pheromone(all_solutions, all_scores)
            print(f"Iteration {iteration+1}, Best Fitness (neg avg dist): {best_score:.4f}")

        return best_solution, -best_score

candidate_stations = [
    (10,10), (20,20), (30,10), (40,40), (50,50),
    (60,60), (70,10), (80,80), (90,90), (25,75)
]

demand_points = [
    (12,15), (22,25), (28,12), (41,39), (48,52),
    (58,62), (69,9), (77,82), (88,91), (24,70),
    (15,85), (35,60), (55,45), (65,30), (75,20)
]

n_stations_to_select = 3

aco = ACOChargingStations(candidate_stations, demand_points, n_stations_to_select)
best_stations, best_avg_distance = aco.run()

print("\nBest charging station locations (indices):", best_stations)
print("Best charging station coordinates:")
for idx in best_stations:
    print(f"  {candidate_stations[idx]}")
print(f"Average distance EVs travel to nearest station: {best_avg_distance:.2f}")


Iteration 1, Best Fitness (neg avg dist): -19.4294
Iteration 2, Best Fitness (neg avg dist): -19.4294
Iteration 3, Best Fitness (neg avg dist): -19.4294
Iteration 4, Best Fitness (neg avg dist): -19.4294
Iteration 5, Best Fitness (neg avg dist): -19.4294
Iteration 6, Best Fitness (neg avg dist): -19.4294
Iteration 7, Best Fitness (neg avg dist): -19.4294
Iteration 8, Best Fitness (neg avg dist): -19.4294
Iteration 9, Best Fitness (neg avg dist): -19.4294
Iteration 10, Best Fitness (neg avg dist): -19.4294
Iteration 11, Best Fitness (neg avg dist): -19.4294
Iteration 12, Best Fitness (neg avg dist): -19.4294
Iteration 13, Best Fitness (neg avg dist): -19.4294
Iteration 14, Best Fitness (neg avg dist): -19.4294
Iteration 15, Best Fitness (neg avg dist): -19.4294
Iteration 16, Best Fitness (neg avg dist): -19.4294
Iteration 17, Best Fitness (neg avg dist): -19.4294
Iteration 18, Best Fitness (neg avg dist): -19.4294
Iteration 19, Best Fitness (neg avg dist): -19.4294
Iteration 20, Best Fi

In [10]:
import numpy as np

class ACO_TSP:
    def __init__(self, dist_matrix, n_ants, n_iterations, decay, alpha=1, beta=2):
        """
        dist_matrix: 2D numpy array of distances between cities
        n_ants: number of ants per iteration
        n_iterations: number of iterations to run
        decay: pheromone evaporation coefficient (0 < decay < 1)
        alpha: pheromone importance
        beta: heuristic importance (usually inverse distance)
        """
        self.dist_matrix = dist_matrix
        self.n_cities = dist_matrix.shape[0]
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.pheromone = np.ones((self.n_cities, self.n_cities))  # initial pheromone

    def run(self):
        best_distance = float('inf')
        best_path = None

        for iteration in range(self.n_iterations):
            all_paths = self.construct_solutions()
            self.evaporate_pheromone()
            self.deposit_pheromone(all_paths)

            distances = [self.path_length(path) for path in all_paths]
            avg_distance = sum(distances) / len(distances)
            min_dist_idx = np.argmin(distances)

            if distances[min_dist_idx] < best_distance:
                best_distance = distances[min_dist_idx]
                best_path = all_paths[min_dist_idx]

            print(f"Iteration {iteration+1}: Best distance = {best_distance}, Average distance = {avg_distance}")

        return best_path, best_distance

    def construct_solutions(self):
        paths = []
        for _ in range(self.n_ants):
            path = self.construct_solution()
            paths.append(path)
        return paths

    def construct_solution(self):
        path = []
        visited = set()

        current_city = np.random.randint(self.n_cities)
        path.append(current_city)
        visited.add(current_city)

        while len(path) < self.n_cities:
            next_city = self.select_next_city(current_city, visited)
            path.append(next_city)
            visited.add(next_city)
            current_city = next_city

        return path

    def select_next_city(self, current_city, visited):
        pheromone = self.pheromone[current_city]
        distances = self.dist_matrix[current_city]

        prob = []
        for city in range(self.n_cities):
            if city not in visited:
                tau = pheromone[city] ** self.alpha
                eta = (1.0 / distances[city]) ** self.beta if distances[city] > 0 else 0
                prob.append(tau * eta)
            else:
                prob.append(0)

        prob = np.array(prob)
        prob_sum = prob.sum()
        if prob_sum == 0:
            # if all prob are zero, choose randomly among unvisited
            choices = [city for city in range(self.n_cities) if city not in visited]
            return np.random.choice(choices)
        prob = prob / prob_sum

        return np.random.choice(range(self.n_cities), p=prob)

    def path_length(self, path):
        length = 0
        for i in range(len(path)):
            length += self.dist_matrix[path[i - 1], path[i]]
        return length

    def evaporate_pheromone(self):
        self.pheromone *= (1 - self.decay)

    def deposit_pheromone(self, paths):
        for path in paths:
            length = self.path_length(path)
            for i in range(len(path)):
                from_city = path[i - 1]
                to_city = path[i]
                self.pheromone[from_city, to_city] += 1.0 / length
                self.pheromone[to_city, from_city] += 1.0 / length  # symmetric

# Example usage:

if __name__ == "__main__":
    # Example distance matrix for 5 cities (symmetric)
    dist_matrix = np.array([
        [0, 23, 9, 10, 7],
        [23, 0, 6, 4, 3],
        [9, 6, 0, 8, 1],
        [10, 4, 8, 0, 6],
        [7, 3, 1, 6, 0]
    ])

    aco = ACO_TSP(dist_matrix, n_ants=10, n_iterations=100, decay=0.1, alpha=1, beta=2)
    best_path, best_distance = aco.run()

    print("Best path:", best_path)
    print("Best distance:", best_distance)


Iteration 1: Best distance = 27, Average distance = 34.4
Iteration 2: Best distance = 27, Average distance = 31.5
Iteration 3: Best distance = 27, Average distance = 29.2
Iteration 4: Best distance = 27, Average distance = 30.9
Iteration 5: Best distance = 27, Average distance = 33.7
Iteration 6: Best distance = 27, Average distance = 27.4
Iteration 7: Best distance = 27, Average distance = 31.1
Iteration 8: Best distance = 27, Average distance = 32.0
Iteration 9: Best distance = 27, Average distance = 28.9
Iteration 10: Best distance = 27, Average distance = 27.0
Iteration 11: Best distance = 27, Average distance = 29.1
Iteration 12: Best distance = 27, Average distance = 27.2
Iteration 13: Best distance = 27, Average distance = 30.5
Iteration 14: Best distance = 27, Average distance = 29.3
Iteration 15: Best distance = 27, Average distance = 29.2
Iteration 16: Best distance = 27, Average distance = 31.3
Iteration 17: Best distance = 27, Average distance = 27.1
Iteration 18: Best dist